---
### Backtesting ML Regression-Based
---

#### I. Load the model

In [31]:
import pickle

In [32]:
with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

---
#### II. Load the data

In [33]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df.head(n=5)

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,55.181126,55.696671,55.027369,55.443420,21220800,1.549151,UP
2016-12-09,56.049416,56.067505,55.289669,55.334891,27349400,0.321666,UP
2016-12-12,56.230289,56.347870,55.823285,55.913730,20198100,1.286169,UP
2016-12-13,56.962929,57.360890,56.293630,56.528788,35718900,-0.478644,DOWN
2016-12-14,56.691578,57.388013,56.555907,56.981005,30352700,-0.159789,DOWN


---
#### III. Backtesting.py Library

Create the Strategy Class.

In [34]:
from backtesting import Backtest, Strategy

In [45]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

Define initial conditions.

In [46]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [47]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

---
#### IV. Run backtesting

In [48]:
results = bt.run()

Interpret backtesting results.

In [49]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2025-04-03 00:00:00
Duration,3038 days 00:00:00
Exposure Time [%],91.822095
Equity Final [$],64160.337125
Equity Peak [$],71321.321388
Commissions [$],2138.229466
Return [%],541.603371


---
#### V. Visualize the strategy performance

In [50]:
bt.plot(filename='backtests/backtesting_regression.html')

GridPlot(id='p2033', ...)